<a href="https://colab.research.google.com/github/dtype2100/TIL/blob/master/soykeyword_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install soynlp

In [2]:
pip install soykeyword

In [3]:
!git clone https://github.com/lovit/textmining_dataset

fatal: destination path 'textmining_dataset' already exists and is not an empty directory.


In [4]:
pip install scikit-learn

In [5]:
!git clone https://github.com/lovit/soykeyword

fatal: destination path 'soykeyword' already exists and is not an empty directory.


In [6]:
pip install soyspacing

In [7]:
import pandas as pd
import numpy as np

In [8]:
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize

In [9]:
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Final/soynlp/newsresult.xlsx')

/usr/local/lib/python3.7/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [10]:
title = df['제목']

In [11]:
#title.to_csv('news_title.txt', index=False)

# Tokenizer

In [12]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import gensim.downloader

In [16]:
pre = list(gensim.downloader.info()['models'].keys())

In [17]:
pre

['fasttext-wiki-news-subwords-300',
 'conceptnet-numberbatch-17-06-300',
 'word2vec-ruscorpora-300',
 'word2vec-google-news-300',
 'glove-wiki-gigaword-50',
 'glove-wiki-gigaword-100',
 'glove-wiki-gigaword-200',
 'glove-wiki-gigaword-300',
 'glove-twitter-25',
 'glove-twitter-50',
 'glove-twitter-100',
 'glove-twitter-200',
 '__testing_word2vec-matrix-synopsis']

In [22]:
glove_vectors = gensim.downloader.load('glove-twitter-25')

[==================================================] 100.0% 104.8/104.8MB downloaded


In [23]:
model = Word2Vec(sentences=title, size=100, window=5, min_count=5, workers=4, sg=0)

In [21]:
#model.build_vocab(title, update=False)

In [35]:
from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer
from soynlp.utils import DoublespaceLineCorpus

file_path = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Final/soynlp/newsresult.xlsx')
corpus = DoublespaceLineCorpus(word_scores, iter_sent=True)

word_extractor = WordExtractor(
    min_frequency=100, # example
    min_cohesion_forward=0.05,
    min_right_branching_entropy=0.0
)

word_extractor.train(title)
word_scores = word_extractor.extract()

cohesion_score = {word:score.cohesion_forward for word, score in word_scores.items()}
ltokenizer = LTokenizer(scores=cohesion_score)

word2vec_corpus = [ltokenizer.tokenize(sent) for sent in title]
word2vec = Word2Vec(word2vec_corpus)

/usr/local/lib/python3.7/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


training was done. used memory 1.069 Gb
all cohesion probabilities was computed. # words = 393
all branching entropies was computed # words = 31193
all accessor variety was computed # words = 31193


In [36]:
word2vec.most_similar('코스피')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('마감', 0.9995260834693909),
 ('증시', 0.9993483424186707),
 ('급등', 0.9993149042129517),
 ('물가', 0.9992461204528809),
 ('세', 0.9992064237594604),
 ('코스', 0.9990102052688599),
 ('뉴욕증시', 0.9989553689956665),
 ('우려', 0.9989262819290161),
 ('국제', 0.9988775849342346),
 ('나스닥', 0.9988505840301514)]

In [41]:
word2vec.most_similar('현대')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('공장', 0.9989898204803467),
 ('인니', 0.9987858533859253),
 ('메타버스', 0.9987660646438599),
 ('미래', 0.9987303614616394),
 ('SK', 0.9987266659736633),
 ('진출', 0.9987058639526367),
 ('최초', 0.9987000226974487),
 ('차,', 0.9986997246742249),
 ('AI', 0.998695969581604),
 ('아이', 0.9986772537231445)]

In [37]:
word_scores['현대차'].cohesion_forward

0.5175716028814108

In [38]:
word_scores['현대차'].right_branching_entropy

2.6055102580250273

In [39]:
word_scores['현대차'].right_accessor_variety

18

# 띄어쓰기 교정

In [169]:
from soyspacing.countbase import CountSpace

In [210]:
corpus_fname = '/content/news_title.txt'
model = CountSpace()
model.train(corpus_fname)

all tags length = 53943 --> 53861, (num_doc = 20000)

In [215]:
verbose=False
mc = 10  # min_count
ft = 0.3 # force_abs_threshold
nt =-0.3 # nonspace_threshold
st = 0.3 # space_threshold

sent = '이건진짜좋은영화 라라랜드진짜좋은영화'

# with parameters
sent_corrected, tags = model.correct(
    doc=sent,
    verbose=verbose,
    force_abs_threshold=ft,
    nonspace_threshold=nt,
    space_threshold=st,
    min_count=mc)

# without parameters
sent_corrected, tags = model.correct(sent)


In [216]:
print(sent_corrected)

이건진짜좋은영화 라라랜드진짜좋은영화


In [220]:
from soynlp.vectorizer import sent_to_word_contexts_matrix

In [221]:
vectorizer = BaseVectorizer(
    tokenizer=tokenizer,
    min_tf=0,
    max_tf=10000,
    min_df=0,
    max_df=1.0,
    stopwords=None,
    lowercase=True,
    verbose=True
)

NameError: ignored

In [218]:
corpus.iter_sent = False
x = vectorizer.fit_transform(title)

NameError: ignored

In [201]:
from soynlp.tokenizer import LTokenizer

scores = {word:score.cohesion_forward for word, score in words.items()}
l_tokenizer = LTokenizer(scores=scores)

l_tokenizer.tokenize("안전성에 문제있는 스마트폰을 휴대하고 탑승할 경우에 압수한다", flatten=False)

[('안전성에', ''),
 ('문제있는', ''),
 ('스마트폰을', ''),
 ('휴대하고', ''),
 ('탑승할', ''),
 ('경우에', ''),
 ('압수한다', '')]

In [202]:
from soynlp.tokenizer import MaxScoreTokenizer

maxscore_tokenizer = MaxScoreTokenizer(scores=scores)
maxscore_tokenizer.tokenize("안전성에문제있는스마트폰을휴대하고탑승할경우에압수한다")

['안전성에문제있는스마', '트폰을휴대하고탑승할', '경우에압수한다']